In [1]:
import pandas as pd
import numpy as np
import operator
import re
from collections import Counter

search_term = 'ziek'
doc = pd.read_csv('sample_dataset.csv', sep=';')

doc.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,ziek src:twitter,2017-09-14 10:26,https://twitter.com/dekoran1/status/9082454469...,NaN,post,NaN,11369.0,dekoran1,11369,131.1,NaN,NaN,@BLIKOPENER333,twitter,NaN
1,ziek src:twitter,2017-09-14 10:25,https://twitter.com/BLIKOPENER333/status/90824...,-,comment,2.0,17592.0,BLIKOPENER333,17592,9.1,NaN,NaN,RT @tikkierechts: Ziek of niet dit is echt een...,twitter,NaN
2,ziek src:twitter,2017-09-14 10:24,https://twitter.com/Jesske3003/status/90824496...,-,comment,19.0,NaN,Jesske3003,856,4.5,NaN,NaN,@tinamieneke @DockxGunter Ja en elkaar wat zie...,twitter,NaN
3,ziek src:twitter,2017-09-14 10:23,https://twitter.com/tinamieneke/status/9082446...,-,comment,19.0,NaN,tinamieneke,1928,25.7,NaN,NaN,"@DockxGunter @Jesske3003 Ik vind dat niet oké,...",twitter,NaN
4,ziek src:twitter,2017-09-14 10:23,https://twitter.com/BeQuanFRIvoedin/status/908...,-,post,NaN,38.0,BeQuanFRIvoedin,38,0.6,NaN,NaN,Hippe huiskip maakte nog nooit zoveel Amerikan...,twitter,NaN


# Cleaning data
Here we do several things:
* remove retweets
* convert text to lowercase

In the context of our research a retweet does not add any relevant data. It actually skews the results since a specific phrasing get's repeated more often than it naturally would.

In [2]:
doc['bericht tekst'] = doc['bericht tekst'].str.lower()

In [3]:
doc = doc[~doc['bericht tekst'].str.contains('rt')]
# doc[~doc['bericht tekst'].str.contains('rt')]
doc.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,ziek src:twitter,2017-09-14 10:26,https://twitter.com/dekoran1/status/9082454469...,NaN,post,NaN,11369.0,dekoran1,11369,131.1,NaN,NaN,@blikopener333,twitter,NaN
3,ziek src:twitter,2017-09-14 10:23,https://twitter.com/tinamieneke/status/9082446...,-,comment,19.0,NaN,tinamieneke,1928,25.7,NaN,NaN,"@dockxgunter @jesske3003 ik vind dat niet oké,...",twitter,NaN
4,ziek src:twitter,2017-09-14 10:23,https://twitter.com/BeQuanFRIvoedin/status/908...,-,post,NaN,38.0,BeQuanFRIvoedin,38,0.6,NaN,NaN,hippe huiskip maakte nog nooit zoveel amerikan...,twitter,NaN
5,ziek src:twitter,2017-09-14 10:19,https://twitter.com/BillyBibixx/status/9082436...,-,post,NaN,775.0,BillyBibixx,775,5.3,NaN,NaN,"ik ga naar huis, dit is echt kut",twitter,NaN
6,ziek src:twitter,2017-09-14 10:18,https://twitter.com/flemmishfan/status/9082436...,-,comment,8.0,NaN,flemmishfan,24,1.3,NaN,NaN,@demorgen mijn idee ook kinderen mogen zich ni...,twitter,NaN


# Defenition of helper function


In [4]:
re_clean = re.compile(r'(https?://\S+|@\S+)')
re_words = re.compile(r'(\w+)')

def clean_text(text: str):
    words = []
    if text:
        text = re_clean.sub(' ', text)
        words = re_words.findall(text)
    return words

In [5]:
all_tweets = pd.read_csv('sample_alltweets.csv', sep=';')
all_tweets.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,src:twitter,2017-06-14 23:59,https://twitter.com/evertdev/status/8751105937...,+,comment,9.0,1214.0,evertdev,1214,1.1,NaN,NaN,RT @JaapTielbeke: Mijn stuk over het onverwach...,twitter,NaN
1,src:twitter,2017-06-14 23:59,https://twitter.com/SoBeOnIt/status/8751105897...,NaN,comment,110.0,236.0,SoBeOnIt,236,0.6,NaN,NaN,RT @RSN_RACAILLE: Conozco a muy pocas personas...,twitter,NaN
2,src:twitter,2017-06-14 23:59,https://twitter.com/spartarn/status/8751105927...,NaN,comment,65.0,50.0,spartarn,50,0.6,NaN,NaN,RT @MLBDraftTracker: 36 (1085): @Tigers select...,twitter,NaN
3,src:twitter,2017-06-14 23:59,https://twitter.com/Neraajz/status/87511059080...,NaN,post,NaN,1548.0,Neraajz,1548,2.9,NaN,NaN,Overmorgen examen en volgende week ook en dan ...,twitter,NaN
4,src:twitter,2017-06-14 23:59,https://twitter.com/pepsi_elwin/status/8751105...,NaN,comment,NaN,NaN,pepsi_elwin,250,1.0,NaN,NaN,@Redwiff that probly requires hacking your swi...,twitter,NaN


# Cleaning data
Here we do several things:
* convert messages with no content with to the message `''`
* remove retweets
* convert to lowercase

The conversion is needed since the `clean_text()` function expects a string.

In [6]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].fillna('')

In [7]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].str.lower()

In [8]:
all_tweets = all_tweets[~all_tweets['bericht tekst'].str.contains('rt')]
all_tweets.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
3,src:twitter,2017-06-14 23:59,https://twitter.com/Neraajz/status/87511059080...,NaN,post,NaN,1548.0,Neraajz,1548,2.9,NaN,NaN,overmorgen examen en volgende week ook en dan ...,twitter,NaN
4,src:twitter,2017-06-14 23:59,https://twitter.com/pepsi_elwin/status/8751105...,NaN,comment,NaN,NaN,pepsi_elwin,250,1.0,NaN,NaN,@redwiff that probly requires hacking your swi...,twitter,NaN
6,src:twitter,2017-06-14 23:59,https://twitter.com/KvW_beach/status/875110593...,NaN,post,NaN,71.0,KvW_beach,71,0.6,NaN,NaN,morgen begint het tornooi voor ons in den haag...,twitter,NaN
7,src:twitter,2017-06-14 23:59,https://twitter.com/rene_fiala/status/87511059...,+,comment,NaN,NaN,rene_fiala,29,0.3,NaN,NaN,@kivicod @stoneemlg wtf hahaha 😂,twitter,NaN
9,src:twitter,2017-06-14 23:59,https://twitter.com/5HOfficialMex/status/87511...,NaN,post,3.0,27834.0,5HOfficialMex,27834,1.7,NaN,NaN,normani en el snapchat de @joshleviworld #3 ht...,twitter,NaN


In [9]:
all_tweets['bericht woorden'] = all_tweets['bericht tekst'].map(clean_text)

counter = Counter()

for words in all_tweets['bericht woorden']:
    counter.update(words)
counter.most_common(250)

[('de', 2969),
 ('in', 2476),
 ('is', 2444),
 ('ik', 2419),
 ('een', 2219),
 ('en', 2174),
 ('van', 1700),
 ('het', 1695),
 ('the', 1639),
 ('je', 1542),
 ('op', 1330),
 ('i', 1328),
 ('a', 1327),
 ('dat', 1257),
 ('to', 1185),
 ('met', 1104),
 ('voor', 1086),
 ('niet', 1086),
 ('of', 1016),
 ('s', 969),
 ('you', 899),
 ('te', 821),
 ('maar', 789),
 ('and', 781),
 ('die', 775),
 ('me', 718),
 ('it', 676),
 ('zijn', 675),
 ('ook', 657),
 ('for', 624),
 ('er', 595),
 ('on', 591),
 ('dan', 551),
 ('aan', 544),
 ('wat', 538),
 ('als', 523),
 ('via', 522),
 ('t', 519),
 ('we', 518),
 ('nog', 489),
 ('was', 485),
 ('wel', 481),
 ('zo', 476),
 ('bij', 472),
 ('om', 468),
 ('heb', 443),
 ('2', 442),
 ('mijn', 438),
 ('ben', 437),
 ('over', 435),
 ('nu', 422),
 ('ze', 422),
 ('my', 421),
 ('this', 419),
 ('dit', 419),
 ('that', 412),
 ('1', 389),
 ('kan', 361),
 ('al', 346),
 ('naar', 341),
 ('so', 340),
 ('geen', 327),
 ('haar', 324),
 ('with', 320),
 ('3', 317),
 ('ja', 316),
 ('m', 311),
 ('

In [10]:
common_words = [word[0] for word in counter.most_common(250)]
common_words = set(common_words)
common_words

{'0',
 '00',
 '1',
 '10',
 '12',
 '14',
 '15',
 '2',
 '2017',
 '23',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'a2',
 'aan',
 'about',
 'af',
 'al',
 'all',
 'alleen',
 'als',
 'altijd',
 'amsterdam',
 'an',
 'and',
 'are',
 'as',
 'at',
 'be',
 'ben',
 'best',
 'bij',
 'blonde',
 'but',
 'by',
 'c',
 'can',
 'd',
 'da',
 'daar',
 'dag',
 'dan',
 'dat',
 'day',
 'de',
 'denk',
 'deze',
 'die',
 'dit',
 'do',
 'doen',
 'don',
 'door',
 'dus',
 'e',
 'echt',
 'een',
 'eens',
 'el',
 'en',
 'er',
 'eu',
 'even',
 'for',
 'foto',
 'from',
 'ga',
 'gaan',
 'gaat',
 'geen',
 'get',
 'gewoon',
 'go',
 'goed',
 'good',
 'great',
 'grote',
 'haar',
 'had',
 'haha',
 'hahaha',
 'hard',
 'has',
 'have',
 'he',
 'heb',
 'hebben',
 'heeft',
 'heel',
 'her',
 'here',
 'het',
 'hier',
 'hij',
 'hoe',
 'how',
 'hun',
 'i',
 'iets',
 'if',
 'ik',
 'in',
 'is',
 'it',
 'ja',
 'jaar',
 'je',
 'jij',
 'jou',
 'jullie',
 'just',
 'kan',
 'keer',
 'know',
 'kom',
 'komt',
 'krijgt',
 'kunnen',
 'la',

In [11]:
blacklisted_words = common_words
blacklisted_words.add(search_term)

In [12]:
doc['bericht woorden'] = doc['bericht tekst'].map(clean_text)
counter = Counter()

for words in doc['bericht woorden']:
    words = set(words)
    filtered_words = words - blacklisted_words
    counter.update(filtered_words)
    
counter.most_common(100)

[('thuis', 271),
 ('bent', 183),
 ('word', 144),
 ('eigen', 136),
 ('kind', 126),
 ('chronisch', 118),
 ('zorg', 115),
 ('financieel', 106),
 ('akzonobel', 106),
 ('ouders', 104),
 ('zich', 100),
 ('risico', 98),
 ('zelf', 93),
 ('directeur', 91),
 ('omdat', 84),
 ('erg', 83),
 ('school', 81),
 ('eigenrisico', 79),
 ('melden', 78),
 ('agenten', 76),
 ('voel', 73),
 ('zit', 73),
 ('bed', 69),
 ('steeds', 65),
 ('maakt', 63),
 ('laat', 62),
 ('beter', 61),
 ('ons', 61),
 ('beetje', 61),
 ('boete', 61),
 ('helemaal', 59),
 ('iemand', 59),
 ('piloten', 56),
 ('werken', 55),
 ('alles', 55),
 ('hoor', 55),
 ('iedereen', 55),
 ('geweest', 53),
 ('ernstig', 53),
 ('dagen', 52),
 ('moeten', 52),
 ('zitten', 52),
 ('3000', 52),
 ('betalen', 51),
 ('geworden', 49),
 ('berlin', 49),
 ('air', 49),
 ('hebt', 48),
 ('hele', 48),
 ('komen', 47),
 ('mag', 47),
 ('beterschap', 46),
 ('politie', 46),
 ('zal', 45),
 ('huis', 44),
 ('zwaar', 43),
 ('werd', 42),
 ('shit', 42),
 ('allemaal', 42),
 ('geld', 4